In [49]:
!pip install fastapi pymongo uvicorn

     ---------------------------------------- 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [6]:
from pymongo import MongoClient
import pandas as pd

# MongoDB에 연결
client = MongoClient("mongodb://localhost:27017/")

# Review 데이터베이스에 접근
db = client["Review"]

# Menu 컬렉션의 데이터를 모두 가져오기
menu_collection = db["Data"]
menu_data = menu_collection.find()

# 데이터를 판다스 DataFrame으로 변환
df = pd.DataFrame(list(menu_data))

print(df)


                             _id        name         id category  \
0       6730695419b7901f8203c82a  자하문로118 들풀   11831738       한식   
1       6730695419b7901f8203c82b  자하문로118 들풀   11831738       한식   
2       6730695419b7901f8203c82c  자하문로118 들풀   11831738       한식   
3       6730695419b7901f8203c82d  자하문로118 들풀   11831738       한식   
4       6730695419b7901f8203c82e  자하문로118 들풀   11831738       한식   
...                          ...         ...        ...      ...   
159611  6730695919b7901f820637a5   써브웨이 중랑역점  327583556     샌드위치   
159612  6730695919b7901f820637a6   써브웨이 중랑역점  327583556     샌드위치   
159613  6730695919b7901f820637a7   써브웨이 중랑역점  327583556     샌드위치   
159614  6730695919b7901f820637a8   써브웨이 중랑역점  327583556     샌드위치   
159615  6730695919b7901f820637a9   써브웨이 중랑역점  327583556     샌드위치   

            menu_name                                description  price  \
0       살살녹는갈비찜(중) 3인                  살살녹는갈비찜 (보통맛) (매운맛)\n소갈비찜  70000   
1            향취자연송이솥밥            

In [4]:
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# MongoDB에 연결하여 Menu 컬렉션 데이터 불러오기
client = MongoClient("mongodb://localhost:27017/")
db = client["Review"]
menu_collection = db["Menu"]
menu_data = pd.DataFrame(list(menu_collection.find()))

# 가격 전처리: "~" 이후의 값을 제거
menu_data['price'] = menu_data['price'].apply(lambda x: x.split("~")[0] if "~" in x else x)

# 숫자 변환 시, 잘못된 형식의 데이터에 대한 예외 처리 추가
def convert_to_float(price):
    try:
        # 쉼표 제거 후 float로 변환
        return float(price.replace(",", "").strip())
    except ValueError:
        # 변환이 불가능할 경우 None을 반환
        return None

# 전처리된 price 컬럼을 숫자로 변환
menu_data['price'] = menu_data['price'].apply(convert_to_float)

# None 값이 있는 경우 제거
menu_data = menu_data.dropna(subset=['price'])

# 데이터 전처리: 'description' 컬럼에 음식명, 설명을 결합
menu_data['description'] = menu_data['name'] # + " " + menu_data['description']

# TF-IDF 벡터화: 음식명, 설명를 포함하여 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(menu_data['description'])

# 사용자 입력 받기
user_food = input("추천받고 싶은 음식명을 입력하세요: ")
user_price = input("가격대를 입력하세요: ")

# 사용자 가격을 float으로 변환
user_price_float = float(user_price.replace(",", "").strip())

# 사용자 입력값을 벡터화하여 데이터에 추가
user_input = f"{user_food} {user_price}"
user_vector = vectorizer.transform([user_input])

# 코사인 유사도 계산
cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

# 유사도와 가격 차이를 데이터프레임에 추가
menu_data['cosine_similarity'] = cosine_similarities
menu_data['price_diff'] = menu_data['price'] - user_price_float   # 가격 차이 (절대값 아님)

# 유사도가 높은 순으로 정렬, 유사도가 같을 경우 가격 차이가 작은 순으로 정렬
sorted_data = menu_data.sort_values(by=['cosine_similarity', 'price_diff'], ascending=[False, True])

# 중복된 restaurant_id 제거하고 상위 10개 추천
unique_recommendations = sorted_data.drop_duplicates(subset=['restaurant_id']).head(10)
print(unique_recommendations)

# 추천된 10개의 식당 ID 목록 생성
restaurant_ids = unique_recommendations['restaurant_id'].tolist()

# Naver_Restaurant_Final 컬렉션에서 해당 ID에 대한 name 조회
restaurant_collection = db["Naver_Restaurant_Final"]
restaurant_names = restaurant_collection.find({"id": {"$in": restaurant_ids}}, {"name": 1, "_id": 0})

# 결과 출력
print("\n추천된 식당 이름들:")
for restaurant in restaurant_names:
    print(restaurant['name'])


추천받고 싶은 음식명을 입력하세요: 짜장면
가격대를 입력하세요: 8000
                             _id name   price description restaurant_id  \
35214   672f1a92a98c4941eaa9898e  짜장면  3000.0         짜장면    1663416492   
161459  672f1a98a98c4941eaab76b3  짜장면  3000.0         짜장면      38488089   
51033   672f1a93a98c4941eaa9c759  짜장면  3500.0         짜장면      11889745   
33866   672f1a92a98c4941eaa9844a  짜장면  4000.0         짜장면      20735508   
51180   672f1a93a98c4941eaa9c7ec  짜장면  4000.0         짜장면      17999623   
51617   672f1a93a98c4941eaa9c9a1  짜장면  4000.0         짜장면    1328375307   
132621  672f1a97a98c4941eaab060d  짜장면  4000.0         짜장면      33937361   
94033   672f1a95a98c4941eaaa6f51  짜장면  4500.0         짜장면      18100560   
156800  672f1a98a98c4941eaab6480  짜장면  4500.0         짜장면    1655097100   
157029  672f1a98a98c4941eaab6565  짜장면  4500.0         짜장면      18083409   

        cosine_similarity  price_diff  
35214                 1.0     -5000.0  
161459                1.0     -5000.0  
51033        

In [6]:
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# MongoDB에 연결하여 Menu 컬렉션 데이터 불러오기
client = MongoClient("mongodb://localhost:27017/")
db = client["Review"]
menu_collection = db["Menu"]
menu_data = pd.DataFrame(list(menu_collection.find()))

# 가격 전처리: "~" 이후의 값을 제거
menu_data['price'] = menu_data['price'].apply(lambda x: x.split("~")[0] if "~" in x else x)

# 숫자 변환 시, 잘못된 형식의 데이터에 대한 예외 처리 추가
def convert_to_float(price):
    try:
        # 쉼표 제거 후 float로 변환
        return float(price.replace(",", "").strip())
    except ValueError:
        # 변환이 불가능할 경우 None을 반환
        return None

# 전처리된 price 컬럼을 숫자로 변환
menu_data['price'] = menu_data['price'].apply(convert_to_float)

# None 값이 있는 경우 제거
menu_data = menu_data.dropna(subset=['price'])

# 데이터 전처리: 'description' 컬럼에 음식명만 결합
menu_data['description'] = menu_data['name']

# TF-IDF 벡터화: 음식명만을 사용하여 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(menu_data['description'])

# 사용자 입력 받기
user_food = input("추천받고 싶은 음식명을 입력하세요: ")
user_price = input("가격대를 입력하세요: ")

# 사용자 가격을 float으로 변환
user_price_float = float(user_price.replace(",", "").strip())

# 사용자 입력값을 벡터화하여 데이터에 추가
user_vector = vectorizer.transform([user_food])

# 코사인 유사도 계산
cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

# 유사도와 가격 차이를 데이터프레임에 추가
menu_data['cosine_similarity'] = cosine_similarities
menu_data['price_diff'] = menu_data['price'] - user_price_float  # 가격 차이

# 가격 차이가 2000 이하인 경우에만 필터링
filtered_data = menu_data[menu_data['price_diff'].abs() <= 2000]

# 유사도가 높은 순으로 정렬, 유사도가 같을 경우 가격 차이가 작은 순으로 정렬
sorted_data = filtered_data.sort_values(by=['cosine_similarity', 'price_diff'], ascending=[False, True])

# 중복된 restaurant_id 제거하고 상위 10개 추천
unique_recommendations = sorted_data.drop_duplicates(subset=['restaurant_id']).head(10)

# 추천된 10개의 식당 ID 목록 생성
restaurant_ids = unique_recommendations['restaurant_id'].tolist()

# Naver_Restaurant_Final 컬렉션에서 해당 ID에 대한 name 조회
restaurant_collection = db["Naver_Restaurant_Final"]
restaurant_names = restaurant_collection.find({"id": {"$in": restaurant_ids}}, {"name": 1, "_id": 0})

# 결과 출력
print("\n추천된 식당 이름들:")
for restaurant in restaurant_names:
    print(restaurant['name'])


추천받고 싶은 음식명을 입력하세요: 짜장면
가격대를 입력하세요: 8000

추천된 식당 이름들:
중국관
태성관
만리장성
래빈
동해루
대륙
부평루
신신원
수유리우동집 남대문점
북경원


In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# MongoDB 연결 설정
client = MongoClient("mongodb://localhost:27017/")
db = client["Review"]
menu_collection = db["Menu"]
restaurant_collection = db["Naver_Restaurant_Final"]

# FastAPI 인스턴스 생성
app = FastAPI()

# 데이터 모델 정의
class RecommendationRequest(BaseModel):
    food_name: str
    price: str

class RecommendationResponse(BaseModel):
    restaurant_names: List[str]

# 가격 변환 함수
def convert_to_float(price):
    try:
        return float(price.replace(",", "").strip())
    except ValueError:
        return None

# 메뉴 데이터를 로드 및 전처리하는 함수
def load_menu_data():
    menu_data = pd.DataFrame(list(menu_collection.find()))
    # 가격 전처리: "~" 이후의 값을 제거하고 float로 변환
    menu_data['price'] = menu_data['price'].apply(lambda x: x.split("~")[0] if "~" in x else x)
    menu_data['price'] = menu_data['price'].apply(convert_to_float)
    menu_data = menu_data.dropna(subset=['price'])
    menu_data['description'] = menu_data['name']
    return menu_data

# 메뉴 데이터 초기 로드
menu_data = load_menu_data()

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(menu_data['description'])

@app.post("/recommend", response_model=RecommendationResponse)
async def recommend(request: RecommendationRequest):
    # 사용자 입력 처리
    user_food = request.food_name
    user_price = convert_to_float(request.price)

    if user_price is None:
        raise HTTPException(status_code=400, detail="Invalid price format")

    # 사용자 입력값 벡터화
    user_vector = vectorizer.transform([user_food])

    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

    # 유사도 및 가격 차이 계산
    menu_data['cosine_similarity'] = cosine_similarities
    menu_data['price_diff'] = menu_data['price'] - user_price

    # 가격 차이가 2000 이하인 경우로 필터링
    filtered_data = menu_data[menu_data['price_diff'].abs() <= 2000]

    # 유사도 기준 정렬, 유사도가 같으면 가격 차이 기준으로 정렬
    sorted_data = filtered_data.sort_values(by=['cosine_similarity', 'price_diff'], ascending=[False, True])

    # 중복 restaurant_id 제거 및 상위 10개 선택
    unique_recommendations = sorted_data.drop_duplicates(subset=['restaurant_id']).head(10)
    restaurant_ids = unique_recommendations['restaurant_id'].tolist()

    # Naver_Restaurant_Final 컬렉션에서 이름 조회
    restaurant_names_cursor = restaurant_collection.find({"id": {"$in": restaurant_ids}}, {"name": 1, "_id": 0})
    restaurant_names = [restaurant['name'] for restaurant in restaurant_names_cursor]

    return RecommendationResponse(restaurant_names=restaurant_names)


In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# MongoDB 연결 설정
client = MongoClient("mongodb://localhost:27017/")
db = client["Review"]
menu_collection = db["Menu"]
restaurant_collection = db["Naver_Restaurant_Final"]

# FastAPI 인스턴스 생성
app = FastAPI()

# 데이터 모델 정의
class RecommendationRequest(BaseModel):
    food_name: str
    price: str

class RecommendationResponse(BaseModel):
    restaurant_names: List[str]

# 가격 변환 함수
def convert_to_float(price):
    try:
        return float(price.replace(",", "").strip())
    except ValueError:
        return None

# 메뉴 데이터를 로드 및 전처리하는 함수
def load_menu_data():
    menu_data = pd.DataFrame(list(menu_collection.find()))
    # 가격 전처리: "~" 이후의 값을 제거하고 float로 변환
    menu_data['price'] = menu_data['price'].apply(lambda x: x.split("~")[0] if "~" in x else x)
    menu_data['price'] = menu_data['price'].apply(convert_to_float)
    menu_data = menu_data.dropna(subset=['price'])
    menu_data['description'] = menu_data['name']
    return menu_data

# 메뉴 데이터 초기 로드
menu_data = load_menu_data()

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(menu_data['description'])

@app.post("/recommend", response_model=RecommendationResponse)
async def recommend(request: RecommendationRequest):
    # 사용자 입력 처리
    user_food = request.food_name
    user_price = convert_to_float(request.price)

    if user_price is None:
        raise HTTPException(status_code=400, detail="Invalid price format")

    # 사용자 입력값 벡터화
    user_vector = vectorizer.transform([user_food])

    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

    # 유사도 및 가격 차이 계산
    menu_data['cosine_similarity'] = cosine_similarities
    menu_data['price_diff'] = menu_data['price'] - user_price

    # 가격 차이가 2000 이하인 경우로 필터링
    filtered_data = menu_data[menu_data['price_diff'].abs() <= 2000]

    # 유사도 기준 정렬, 유사도가 같으면 가격 차이 기준으로 정렬
    sorted_data = filtered_data.sort_values(by=['cosine_similarity', 'price_diff'], ascending=[False, True])

    # 중복 restaurant_id 제거 및 상위 10개 선택
    unique_recommendations = sorted_data.drop_duplicates(subset=['restaurant_id']).head(10)
    restaurant_ids = unique_recommendations['restaurant_id'].tolist()

    # Naver_Restaurant_Final 컬렉션에서 이름 조회
    restaurant_names_cursor = restaurant_collection.find({"restaurant_id": {"$in": restaurant_ids}}, {"name": 1, "_id": 0})
    restaurant_names = [restaurant['name'] for restaurant in restaurant_names_cursor]

    return RecommendationResponse(restaurant_names=restaurant_names)


In [ ]:
!uvicorn recommendation_server:app --reload

In [ ]:
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# MongoDB에 연결하여 Menu 컬렉션 데이터 불러오기
client = MongoClient("mongodb://localhost:27017/")
db = client["Review"]
menu_collection = db["Data"]
menu_data = pd.DataFrame(list(menu_collection.find()))

# 데이터 전처리: 'description' 컬럼에 음식명과 설명을 결합
menu_data['description'] = menu_data['name'] # + " " + menu_data['description']

# TF-IDF 벡터화: 음식명과 설명을 포함하여 벡터화
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(menu_data['description'])

# 사용자 입력 받기
user_food = input("추천받고 싶은 음식명을 입력하세요: ")
user_price = input("가격대를 입력하세요: ")

# 사용자 가격을 float으로 변환
user_price_float = float(user_price.replace(",", "").strip())

# 사용자 입력값을 벡터화하여 데이터에 추가
user_input = f"{user_food} {user_price}"
user_vector = vectorizer.transform([user_input])

# 코사인 유사도 계산
cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

# 유사도와 가격 차이를 데이터프레임에 추가
menu_data['cosine_similarity'] = cosine_similarities
menu_data['price_diff'] = menu_data['price'] - user_price_float  # 가격 차이 (절대값 아님)

# 유사도가 높은 순으로 정렬, 유사도가 같을 경우 가격 차이가 작은 순으로 정렬
sorted_data = menu_data.sort_values(by=['cosine_similarity', 'price_diff'], ascending=[False, True])

# 중복된 식당 ID를 제거하고 상위 10개 추천
unique_recommendations = sorted_data.drop_duplicates(subset=['id']).head(10)
print(unique_recommendations)

# 추천된 10개의 식당 ID 목록 생성
restaurant_ids = unique_recommendations['id'].tolist()

# Naver_Restaurant_Final 컬렉션에서 해당 ID에 대한 name 조회
restaurant_collection = db["Naver_Restaurant_Final"]
restaurant_names = restaurant_collection.find({"id": {"$in": restaurant_ids}}, {"name": 1, "_id": 0})

# 결과 출력
print("\n추천된 식당 이름들:")
for restaurant in restaurant_names:
    print(restaurant['name'])


In [8]:
!curl -X 'POST' \
  'http://127.0.0.1:8000/recommend_restaurants' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "food_name": "불고기",
  "price": 15000
}'


IndentationError: unexpected indent (389830866.py, line 2)